is blocking (like exclude based on length difference an option?)
for the ones with low cosine similarity, just manually search?

Parameter k = how many neighbors you return, Parameter ef = How many neighbors you explore


In [1]:
import pandas as pd
# read test_pairs_all.parquet in data folder
df = pd.read_parquet('data/processed/test_pairs_all.parquet')
# take only first 10000 rows
df = df.head(50000)
print(df.head())

  fraudulent_name   real_name  label
0      mekwajisyo  meiwajisyo    1.0
1      meiawijsyo  meiwajisyo    1.0
2     meiwa-ijsoy  meiwajisyo    1.0
3      męiwajisyo  meiwajisyo    1.0
4      meįwąjisyo  meiwajisyo    1.0


In [2]:
df_real = df[['real_name']].drop_duplicates().reset_index(drop=True)
print(df_real.head())

           real_name
0         meiwajisyo
1   webfactorydesign
2             125wyt
3           aeonbank
4  filmesonlineagora


In [3]:
print(len(df_real)*len(df))

277800000


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    analyzer="char",
    ngram_range=(3, 5),
    min_df=1
)

X_real = vectorizer.fit_transform(df_real["real_name"]).astype("float32")
X_real = X_real.toarray()


In [ ]:
import hnswlib

dim = X_real.shape[1]

index = hnswlib.Index(space="cosine", dim=dim)
index.init_index(
    max_elements=len(X_real),
    ef_construction=200,
    M=16
)

index.add_items(X_real, ids=df_real.index.to_numpy())
index.set_ef(int(0.1*len(X_real)))

In [20]:
from rapidfuzz.distance import Levenshtein

def lev_sim(a, b):
    return 1 - Levenshtein.distance(a, b) / max(len(a), len(b))

X_query = vectorizer.transform(df["real_name"]).astype("float32")
X_query = X_query.toarray()

In [18]:
K = int(0.05*len(X_real))

results = []

for i, q_vec in enumerate(X_query):
    labels, distances = index.knn_query(q_vec, k=K)
    candidate_ids = labels[0]

    query_norm = df.loc[i, "real_name"]

    scored = []
    for cid in candidate_ids:
        real_norm = df_real.loc[cid, "real_name"]
        score = lev_sim(query_norm, real_norm)
        scored.append((cid, score))

    best_cid, best_score = max(scored, key=lambda x: x[1])

    results.append({
        "query_name": df.loc[i, "fraudulent_name"],
        "actual_real_name": df.loc[i, "real_name"],
        "best_real_name": df_real.loc[best_cid, "real_name"],
        "lev_similarity": best_score
    })


In [19]:
def decision(score):
    if score >= 0.90:
        return "block"
    elif score >= 0.80:
        return "review"
    return "allow"

out = pd.DataFrame(results)
out["decision"] = out["lev_similarity"].apply(decision)

In [20]:
print(out.head(20))

           query_name  actual_real_name    best_real_name  lev_similarity  \
0          mekwajisyo        meiwajisyo        meiwajisyo             1.0   
1          meiawijsyo        meiwajisyo        meiwajisyo             1.0   
2         meiwa-ijsoy        meiwajisyo        meiwajisyo             1.0   
3          męiwajisyo        meiwajisyo        meiwajisyo             1.0   
4          meįwąjisyo        meiwajisyo        meiwajisyo             1.0   
5          meiwàjišÿo        meiwajisyo        meiwajisyo             1.0   
6          meiwajiryò        meiwajisyo        meiwajisyo             1.0   
7          leiwâjiŝyo        meiwajisyo        meiwajisyo             1.0   
8         rneiwaijsyô        meiwajisyo        meiwajisyo             1.0   
9    wdbfactorydesign  webfactorydesign  webfactorydesign             1.0   
10  webfacttorydesgin  webfactorydesign  webfactorydesign             1.0   
11   webfictorydesgkn  webfactorydesign  webfactorydesign             1.0   

In [21]:
# print accuracy as percentage of rows where actual_real_name == best_real_name
accuracy = (out["actual_real_name"] == out["best_real_name"]).mean()
print(f"Accuracy: {accuracy:.4%}")

Accuracy: 96.3100%


In [22]:
# print rows where actual_real_name != best_real_name
mismatches = out[out["actual_real_name"] != out["best_real_name"]]
print(mismatches)

     query_name actual_real_name best_real_name  lev_similarity decision
1395        z6r               z6           nzxt        0.250000    allow
1396        zbc               z6           nzxt        0.250000    allow
1397       zbdi               z6           nzxt        0.250000    allow
1398         ẕ6               z6           nzxt        0.250000    allow
1399         ẓ6               z6           nzxt        0.250000    allow
...         ...              ...            ...             ...      ...
9013         ƀɱ               bm            fbo        0.333333    allow
9014         ƀɱ               bm            fbo        0.333333    allow
9015        bɱm               bm            fbo        0.333333    allow
9016         ɱƀ               bm            fbo        0.333333    allow
9017          ɍ               bm            fbo        0.333333    allow

[369 rows x 5 columns]


In [10]:
import time
import numpy as np

from rapidfuzz.distance import Levenshtein

number = 0

def lev_sim(a, b):
    return 1 - Levenshtein.distance(a, b) / max(len(a), len(b))

real_names = df_real["real_name"].tolist()

start = time.perf_counter()

brute_results = []
for q in df["fraudulent_name"]:
    best_score = -1
    best_name = None

    for name in real_names:
        score = lev_sim(q, name)
        if score > best_score:
            best_score = score
            best_name = name
        number += 1

brute_time = time.perf_counter() - start

print(f"Brute force time: {brute_time:.3f} seconds")
print(len(df_real))
print(number)


Brute force time: 101.745 seconds
5556
277800000


In [1]:
import hnswlib
import time

from sklearn.feature_extraction.text import TfidfVectorizer

import pandas as pd
# read test_pairs_all.parquet in data folder
df = pd.read_parquet('data/processed/test_pairs_all.parquet')
# take only first 10000 rows
df = df.head(50000)
print(df.head())

df_real = df[['real_name']].drop_duplicates().reset_index(drop=True)
print(df_real.head())

vectorizer = TfidfVectorizer(
    analyzer="char",
    ngram_range=(3, 5),
    min_df=1
)

X_real = vectorizer.fit_transform(df_real["real_name"]).astype("float32")
X_real = X_real.toarray()


K = 1

dim = X_real.shape[1]

index = hnswlib.Index(space="cosine", dim=dim)
index.init_index(
    max_elements=len(X_real),
    ef_construction=5,
    M=10
)

from rapidfuzz.distance import Levenshtein

def lev_sim(a, b):
    return 1 - Levenshtein.distance(a, b) / max(len(a), len(b))

X_query = vectorizer.transform(df["real_name"]).astype("float32")
X_query = X_query.toarray()

start = time.perf_counter()

index.add_items(X_real, ids=df_real.index.to_numpy())

index.set_ef(5)

results = []

labels, distances = index.knn_query(X_query, k=K)

brute_time = time.perf_counter() - start

print(f"HNSW time: {brute_time:.3f} seconds")


  fraudulent_name   real_name  label
0      mekwajisyo  meiwajisyo    1.0
1      meiawijsyo  meiwajisyo    1.0
2     meiwa-ijsoy  meiwajisyo    1.0
3      męiwajisyo  meiwajisyo    1.0
4      meįwąjisyo  meiwajisyo    1.0
           real_name
0         meiwajisyo
1   webfactorydesign
2             125wyt
3           aeonbank
4  filmesonlineagora
HNSW time: 66.882 seconds
